#**Problem 1: Skin Lesion Image Classification**#
The dataset is the ISIC2018 task 3, including 10,015 training images, 193 validation images, and
1,512 test images with seven diseases.

#### **Install Libraries**

In [ ]:
!pip install torch
!pip install torchvision
!pip install pandas
!pip install scikit-learn
!pip install tqdm

#### **Check for GPU if using google colab**

In [ ]:
!nvidia-smi

#### **Mount Drive if needed**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### **Import Libraries**

In [4]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch.nn as nn
from torchvision import models
import torch.optim as optim

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

#### **Define the dataloader and load the data**

In [5]:
class SkinLesionDataset(Dataset):
    def __init__(self, images_dir, labels_file, transform=None):
        self.images_dir = images_dir
        self.labels_df = pd.read_csv(labels_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # Get the image file name
        image_name = self.labels_df.iloc[idx, 0]
        image_path = os.path.join(self.images_dir, image_name)
        image_path = f"{image_path}.jpg"


        # Load the image
        image = Image.open(image_path).convert("RGB")

        # Get the labels and convert to single class index
        labels = self.labels_df.iloc[idx, 1:].values.astype(float)  # Select all label columns
        class_idx = labels.argmax()  # Get the index of the max value

        if self.transform:
            image = self.transform(image)

        return image, class_idx  # Return the image and the class index

# Directory and file paths
train_images_dir = '/content/drive/MyDrive/ISIC2018_Task3_Training_Input' # PATH to training images folder
train_labels_file = os.path.join(train_images_dir, 'ISIC2018_Task3_Training_GroundTruth.csv') # training labels file
val_images_dir = '/content/drive/MyDrive/ISIC2018_Task3_Validation_Input' # PATH to validation images folder
val_labels_file = os.path.join(val_images_dir, 'ISIC2018_Task3_Validation_GroundTruth.csv')# validation labels file
test_images_dir = '/content/drive/MyDrive/ISIC2018_Task3_Test_Input' # PATH to testing images folder
test_labels_file = os.path.join(test_images_dir, 'ISIC2018_Task3_Test_GroundTruth.csv')# testing labels file

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create dataset and dataloader
train_dataset = SkinLesionDataset(train_images_dir, train_labels_file, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = SkinLesionDataset(val_images_dir, val_labels_file, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=25, shuffle=True)
test_dataset = SkinLesionDataset(test_images_dir, test_labels_file, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

####***OPTIONAL modules (DO NOT RUN)***####
function to calculate class weights. This is not used in our training configurations.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import class_weight

# Load the labels and count class instances
def compute_class_weights(labels_file):
    labels_df = pd.read_csv(labels_file)
    all_labels = labels_df.values[:, 1:]  # Assuming labels are in the second column onward
    class_indices = all_labels.argmax(axis=1)  # Get the index of the max value for each row
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(class_indices), y=class_indices)
    return torch.tensor(class_weights, dtype=torch.float).to(device)


# Compute class weights
class_weights = compute_class_weights(train_labels_file)

print("Class Weights:", class_weights)

In [ ]:
import torch
import torch.nn as nn

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        # Calculate the Euclidean distance between the two outputs
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)

        # Compute the contrastive loss
        loss = (1 - label) * torch.pow(euclidean_distance, 2) + \
               (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        return loss.mean()

#### **CONFIGURATION 1**

In [27]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        correct = 0
        total = 0

        # setup training progress bar
        train_loop = tqdm(enumerate(train_loader), total=len(train_loader), leave= False)

        for batch_idx, (images, labels) in train_loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)

            # recording loss
            loss = criterion(outputs, labels)
            batch_train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            train_running_loss += loss.item()

            # recording accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            batch_correct = (predicted == labels).sum().item()
            correct += batch_correct
            acc = 100 * batch_correct/labels.size(0)
            batch_train_acc.append(acc)

            # update progress bar
            train_loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            train_loop.set_postfix(loss=loss.item(), acc=acc)



        # Calculate training accuracy
        epoch_train_accuracy = 100 * correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        val_running_loss = 0.0

        # setup validation progress bar
        val_loop = tqdm(enumerate(val_loader), total=len(val_loader), leave= False)

        with torch.no_grad():
            for batch_idx, (images, labels) in val_loop:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)

                 # recording loss
                loss = criterion(outputs, labels)
                batch_val_loss.append(loss.item())
                val_running_loss += loss.item()

                # recording accuracy
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                batch_correct = (predicted == labels).sum().item()
                val_correct += batch_correct
                acc = 100 * batch_correct/labels.size(0)
                batch_val_acc.append(acc)

                # update progress bar
                val_loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
                val_loop.set_postfix(loss=loss, acc=acc)

        # Calculate validation accuracy
        epoch_val_accuracy = 100 * val_correct / val_total

        # Print progress
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_running_loss/len(train_loader):.4f}, '
              f'Train Accuracy: {epoch_train_accuracy:.2f}%, '
              f'Val Loss: {val_running_loss/len(val_loader):.4f}, '
              f'Val Accuracy: {epoch_val_accuracy:.2f}%')

        # Save the model if the validation accuracy is the best
        if epoch_val_accuracy > best_val_accuracy:
            best_val_accuracy = epoch_val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'Saved model with accuracy: {best_val_accuracy:.2f}%')


In [28]:
# CONFIGURATION 1

# Initialise loss and accuracy lists
batch_train_loss = [] # Training loss
batch_val_loss = [] # Validation loss
batch_train_acc = [] # Training accuracy
batch_val_acc = [] # Validation accuracy

import torchvision

# Load pre-trained ResNet50
model = models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT,progress=True)

# Modify the final layer for multi-label classification
num_classes = 7  # Number of skin diseases
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=15)

#### **CONFIGURATION 2 Note: If you run this, the previous configuration data will be replaced**

Run the evaluation metrics in the next section first before running configuration 2 if you want to view results for configuration 1

In [6]:
from tqdm import tqdm
import torch
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # Probability of true class
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss


In [7]:
def train_focal_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        train_correct = 0
        train_total = 0

        # Training loop
        train_loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)

        for batch_idx, (images, labels) in train_loop:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Calculate Focal Loss
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Update running loss
            train_running_loss += loss.item()

            # Calculate accuracy
            _, preds = torch.max(outputs, 1)  # Get predicted class
            correct = (preds == labels).sum().item()
            train_correct += correct
            total = labels.size(0)
            train_total += total
            acc = (correct / total) * 100

            batch_train_loss.append(loss.item())
            batch_train_acc.append(acc)

            # Update progress bar
            train_loop.set_description(f"Epoch [{epoch + 1}/{num_epochs}]")
            train_loop.set_postfix(loss=loss.item(), acc=acc)

        # Calculate training accuracy
        epoch_train_accuracy = 100 * train_correct / train_total
        avg_train_loss = train_running_loss / len(train_loader)

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        val_running_loss = 0.0

        val_loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=False)

        with torch.no_grad():
            for batch_idx, (images, labels) in val_loop:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)

                # Calculate Focal Loss
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                # Calculate accuracy
                _, preds = torch.max(outputs, 1)
                correct = (preds == labels).sum().item()
                val_correct += correct
                val_total += labels.size(0)
                acc = (correct / labels.size(0)) * 100

                batch_val_loss.append(loss.item())
                batch_val_acc.append(acc)

                # Update progress bar
                val_loop.set_description(f"Epoch [{epoch + 1}/{num_epochs}]")
                val_loop.set_postfix(loss=loss.item(), acc=acc)

        # Calculate validation accuracy
        epoch_val_accuracy = 100 * val_correct / val_total
        avg_val_loss = val_running_loss / len(val_loader)

        # Save the model if the validation accuracy is the best
        if epoch_val_accuracy > best_val_accuracy:
            best_val_accuracy = epoch_val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
            print(f'Saved model with validation accuracy: {best_val_accuracy:.2f}%')

        # Print progress
        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, '
              f'Train Accuracy: {epoch_train_accuracy:.2f}%, '
              f'Val Loss: {avg_val_loss:.4f}, '
              f'Val Accuracy: {epoch_val_accuracy:.2f}%')

In [ ]:
import torchvision.models as models
import torch.nn as nn

# Initialise loss and accuracy lists
batch_train_loss = [] # Training loss
batch_val_loss = [] # Validation loss
batch_train_acc = [] # Training accuracy
batch_val_acc = [] # Validation accuracy

# Load the model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT, progress=True)

# Number of classes for your specific task (e.g., skin diseases)
num_classes = 7  # Adjust based on your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify final layer for multi-class classification

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = FocalLoss(alpha=1.0, gamma=2.0)  # Use Focal Loss instead of Contrastive Loss

# Call the training function
train_focal_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

#### **Evaluation Metrics**


In [ ]:
# Load the best performing model based on the validation set
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
from sklearn.metrics import classification_report
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    print(classification_report(all_labels, all_preds, target_names=val_dataset.labels_df.columns[1:], zero_division=0))

evaluate_model(model, test_loader)

#### **Plot loss and accuracy curves**

In [ ]:
import matplotlib.pyplot as plt

train_index = []
val_index = []
num_epochs = 10 # Adjust number of epochs if needed config 2 - 15, config 1 - 10

train_batch = 313
val_batch = 8

for i in range(1,num_epochs*train_batch + 1):
  train_index.append(i)

for i in range(1,num_epochs*val_batch+1):
  val_index.append(i*(train_batch/val_batch))
# Create a figure with subplots
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(train_index, batch_train_loss, label='Training Loss', color='blue')
plt.title('Loss per batch')
plt.xlabel('Batch')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(val_index, batch_val_loss, label='Validation Loss', color='orange')
plt.title('Loss per batch')
plt.xlabel('Batch')
plt.ylabel('Loss')
plt.legend()
plt.grid()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_index, batch_train_acc, label='Training Accuracy', color='green')
plt.title('Accuracy per batch')
plt.xlabel('Batch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(val_index, batch_val_acc, label='Validation Accuracy', color='red')
plt.title('Accuracy per batch')
plt.xlabel('Batch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid()

# Show the plots
plt.tight_layout()
plt.show()

#### **Plot Multiclass ROC curves**

In [17]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

def obtain_predictions(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # Get predicted probabilities using softmax
            probabilities = torch.softmax(outputs, dim=1)
            all_probs.append(probabilities.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

            # Get predicted classes
            _, predicted = torch.max(outputs.data, 1)
            all_preds.append(predicted.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    all_probs = np.concatenate(all_probs)

    return all_labels, all_preds, all_probs

# Load the best model and evaluate
# model.load_state_dict(torch.load('best_model.pth', map_location=torch.device('cpu')))
all_labels, all_preds, all_probs = obtain_predictions(model, test_loader)

In [ ]:
def plot_multiclass_roc(all_labels, all_probs, num_classes, class_names):
    from sklearn.preprocessing import label_binarize
    from sklearn.metrics import roc_curve, auc

    # Binarize the labels
    y_bin = label_binarize(all_labels, classes=range(num_classes))

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], all_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot ROC curves
    plt.figure()
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'{class_names[i]} (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multiclass ROC Curve')
    plt.legend(loc='lower right')
    plt.show()

# Number of classes (should match your dataset)
num_classes = 7  # Adjust this to your specific number of classes

# Plot the multiclass ROC curves
class_names = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
plot_multiclass_roc(all_labels, all_probs, num_classes,class_names)

#### **Plot Gradient Class Activation Maps**

In [19]:
import torch
import torch.nn.functional as F
import torchvision.models as models

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        # Register hooks for the target layer
        self.target_layer.register_forward_hook(self.save_activation)
        self.target_layer.register_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate_cam(self, class_idx):
        # Get the weights from the gradients
        weights = F.adaptive_avg_pool2d(self.gradients, 1)
        cam = weights.view(-1) @ self.activations.view(self.activations.size(1), -1)
        cam = cam.view(self.activations.size(2), self.activations.size(3))

        # Apply ReLU
        cam = F.relu(cam)

        # Normalize the CAM
        cam = cam - cam.min()
        cam /= cam.max()

        return cam.cpu().data.numpy()



In [ ]:
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
import numpy as np

# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

def plot_gradcam(model, image_paths, target_classes):
    model.eval()
    grad_cam = GradCAM(model, model.layer4)  # Call GradCAM, you can change the layer to check other layers

    # Create a figure for the 6 subplots
    plt.figure(figsize=(20, 8))  # Adjust size as needed

    for idx, (img_path, target_class) in enumerate(zip(image_paths, target_classes)):
        input_tensor = preprocess_image(img_path).to(device)

        # Forward pass
        output = model(input_tensor)
        pred_class = output.argmax(dim=1).item()

        # Uncomment to output predicted class
        # print(pred_class)
        # Backward pass
        model.zero_grad()
        output[0][target_class].backward()

        # Generate Grad-CAM
        cam = grad_cam.generate_cam(target_class)

        # Load original image for visualization
        original_image = cv2.imread(img_path)
        original_image = cv2.resize(original_image, (224, 224))  # Resize to match model input

        # Convert CAM to heatmap
        heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

        # Resize heatmap to match original image dimensions
        heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))

        # Normalize heatmap
        heatmap = heatmap / 255.0

        # Overlay heatmap on the original image
        superimposed_img = heatmap + original_image / 255.0
        superimposed_img /= superimposed_img.max()  # Normalize to [0, 1]

        # Define subplot index
        subplot_index = idx * 3

        # Plot the images
        plt.subplot(len(image_paths)//2 + 1, 6, subplot_index + 1)  # Original Image
        plt.imshow(original_image)
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(len(image_paths)//2 + 1, 6, subplot_index + 2)  # Grad-CAM
        plt.imshow(cam, cmap='jet', alpha=0.5)
        plt.title('Grad-CAM')
        plt.axis('off')

        plt.subplot(len(image_paths)//2 + 1, 6, subplot_index + 3)  # Overlay
        plt.imshow(superimposed_img)
        plt.title('Overlay')
        plt.axis('off')

    plt.tight_layout()  # Adjusts spacing to prevent overlap
    plt.show()  # Show all plots together
# Example image paths and target classes

# class_0 melanoma
# image_paths = [
#     'ISIC2018_Task3_Test_Input/ISIC_0034595.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034650.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034657.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034687.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034737.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034764.jpg'
# ]

# class_1 nv
# image_paths = [
#     'ISIC2018_Task3_Test_Input/ISIC_0034528.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034532.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034533.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034534.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034537.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034540.jpg'
# ]

# class_2 bcc
# image_paths = [
#     'ISIC2018_Task3_Test_Input/ISIC_0034554.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034606.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034679.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034682.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034695.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034712.jpg'
# ]

# class_3 akiec
# image_paths = [
#     'ISIC2018_Task3_Test_Input/ISIC_0034579.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035149.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035210.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035372.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035376.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035348.jpg'
# ]

# class_4 bkl
# image_paths = [
#     'ISIC2018_Task3_Test_Input/ISIC_0034526.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034531.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034535.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034570.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034578.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0034591.jpg'
# ]

# class_5 df
# image_paths = [
#     'ISIC2018_Task3_Test_Input/ISIC_0035239.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035552.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035821.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035844.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035904.jpg',
#     'ISIC2018_Task3_Test_Input/ISIC_0035940.jpg'
# ]

# class_6 VASC
image_paths = [
    'ISIC2018_Task3_Test_Input/ISIC_0034911.jpg',
    'ISIC2018_Task3_Test_Input/ISIC_0034739.jpg',
    'ISIC2018_Task3_Test_Input/ISIC_0034805.jpg',
    'ISIC2018_Task3_Test_Input/ISIC_0034954.jpg',
    'ISIC2018_Task3_Test_Input/ISIC_0035194.jpg',
    'ISIC2018_Task3_Test_Input/ISIC_0034896.jpg'
]

# Target classes for each image (index based on your class names)

target_classes = [6,6,6,6,6,6] # Adjust based on the class indices (0 - MEL, 1 - NV, 2 - BCC, 3 - AKIEC, 4 - BKL, 5 - DF, 6 - VASC)

# Class names for reference
class_names = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']

# Plot Grad-CAMs
plot_gradcam(model, image_paths, target_classes)